In [ ]:
import numpy as np
import pandas as pd

from datetime import date

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
train_offline_data_type = {'User_id': np.int64, 'Merchant_id': np.int64, 'Coupon_id': object,
              'Discount_rate': object, 'Distance' :object, 'Date_received': object,
              'Date': object}
test_offline_data_type = {'User_id': np.int64, 'Merchant_id': np.int64, 'Coupon_id': object,
              'Discount_rate': object, 'Distance' :object, 'Date_received': object}
train_online_data_type = {'User_id': np.int64, 'Merchant_id': np.int64, 'Action': np.int64,
              'Coupon_id': object, 'Discount_rate': object, 'Distance' :object,
              'Date_received': object}
              
test_offline = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/ccf_offline_stage1_test_revised.csv', dtype=test_offline_data_type)
train_offline = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/ccf_offline_stage1_train.csv', dtype=train_offline_data_type)
train_online = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/ccf_online_stage1_train.csv', dtype=train_online_data_type)
submission = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/sample_submission.csv')

In [ ]:
print('test_offline dataset has {} rows and {} columns.'.format(test_offline.shape[0],test_offline.shape[1]))
print('train_offline dataset has {} rows and {} columns.'.format(train_offline.shape[0],train_offline.shape[1]))
print('train_online dataset has {} rows and {} columns.'.format(train_online.shape[0],train_online.shape[1]))
print('submission dataset has {} rows and {} columns.'.format(submission.shape[0],submission.shape[1]))

test_offline dataset has 113640 rows and 6 columns.
train_offline dataset has 1754884 rows and 7 columns.
train_online dataset has 11429826 rows and 7 columns.
submission dataset has 13 rows and 4 columns.


## 优惠券特征

提取折扣率、使用所需满多少金额

In [ ]:
# convert Discount_rate and Distance

def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

def processData(df):
    
    # convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].apply(convertRate)
    df['discount_man'] = df['Discount_rate'].apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].apply(getDiscountType)
    print(df['discount_rate'].unique())
    
    # convert distance
    df['distance'] = df['Distance'].replace(np.nan, -1).astype(int)
    print(df['distance'].unique())
    return df

train_offline['Discount_rate'] = train_offline['Discount_rate'].astype(str)
train_offline = processData(train_offline)
test_offline['Discount_rate'] = test_offline['Discount_rate'].astype(str)
test_offline = processData(test_offline)

[       nan 0.86666667 0.95       0.9        0.83333333 0.8
 0.5        0.85       0.75       0.66666667 0.93333333 0.7
 0.6        0.96666667 0.98       0.99       0.975      0.33333333
 0.2        0.4       ]
[ 0  1 -1  2 10  4  7  9  3  5  6  8]
[0.83333333 0.9        0.96666667 0.8        0.95       0.75
 0.98       0.5        0.86666667 0.6        0.66666667 0.7
 0.85       0.33333333 0.94       0.93333333 0.975      0.99      ]
[ 1 -1  5  2  0 10  3  6  7  4  9  8]


提取星期特征

In [ ]:
def getWeekday(row):
    if row == 'nan':
        return row
    else:
        return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1

train_offline['weekday'] = train_offline['Date_received'].astype(str).apply(getWeekday)
test_offline['weekday'] = test_offline['Date_received'].astype(str).apply(getWeekday)

# weekday_type :  周六和周日为1，其他为0
train_offline['weekday_type'] = train_offline['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )
test_offline['weekday_type'] = test_offline['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )

In [ ]:
# change weekday to one-hot encoding 
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

tmpdf = pd.get_dummies(train_offline['weekday'].replace('nan', np.nan))
tmpdf.columns = weekdaycols
train_offline[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(test_offline['weekday'].replace('nan', np.nan))
tmpdf.columns = weekdaycols
test_offline[weekdaycols] = tmpdf

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


数据标注

Date_received == 'null': y = -1

Date != 'null' & Date-Date_received <= 15: y = 1

Otherwise: y = 0

In [ ]:
def label(row):
    if row['Date_received'] is np.nan:
        return -1
    if row['Date'] is not np.nan:
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0
train_offline['label'] = train_offline.apply(label, axis = 1)

使用券时间间隔

In [ ]:
def getTimeInterval(row):
    if row['Date_received'] is np.nan:
        return np.nan
    if row['Date'] is not np.nan:
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        return td.days
    return np.nan
train_offline['Purchase_Interval'] = train_offline.apply(getTimeInterval, axis = 1)
train_online['Purchase_Interval'] = train_online.apply(getTimeInterval, axis = 1)

# 特征工程

## 用户特征

In [ ]:
# key of user
u = train_offline[['User_id']].copy().drop_duplicates()

# 某一用户领取线下优惠券的次数 u1
u1 = train_offline[~train_offline['Date_received'].isna()][['User_id']].copy()
u1['u_coupon_count'] = 1
u1 = u1.groupby(['User_id'], as_index = False).count()
# 某一用户线下购买的次数 u2
u2 = train_offline[~train_offline['Date'].isna()][['User_id']].copy()
u2['u_buy_count'] = 1
u2 = u2.groupby(['User_id'], as_index = False).count()
# 某一用户使用线下优惠券的次数
u3 = train_offline[((~train_offline['Date'].isna()) & (~train_offline['Date_received'].isna()))][['User_id']].copy()
u3['u_buy_with_coupon'] = 1
u3 = u3.groupby(['User_id'], as_index = False).count()
# 某一用户购买过的线下商店数
u4 = train_offline[~train_offline['Date'].isna()][['User_id', 'Merchant_id']].copy()
u4 = u4.groupby(['User_id'], as_index = False).aggregate('nunique')
u4.rename(columns = {'Merchant_id':'u_merchant_count'}, inplace = True)
# 某个用户购买距离的各种统计值
utmp = train_offline[(~train_offline['Date'].isna()) & (~train_offline['Date_received'].isna())][['User_id', 'distance']].copy()
utmp.replace(-1, np.nan, inplace = True)
u5 = utmp.groupby(['User_id'], as_index = False).min()
u5.rename(columns = {'distance':'u_min_distance'}, inplace = True)
u6 = utmp.groupby(['User_id'], as_index = False).max()
u6.rename(columns = {'distance':'u_max_distance'}, inplace = True)
u7 = utmp.groupby(['User_id'], as_index = False).mean()
u7.rename(columns = {'distance':'u_mean_distance'}, inplace = True)
u8 = utmp.groupby(['User_id'], as_index = False).median()
u8.rename(columns = {'distance':'u_median_distance'}, inplace = True)
u8.head(2)
# 某个用户线下优惠券使用率（使用/领取）
u9 = u1.copy()
u9 = u9.merge(u3, on='User_id', how='left')
u9['u_buy_with_coupon'] = u9['u_buy_with_coupon'].fillna(0)
u9['coupon_use_rate'] = u9['u_buy_with_coupon']/u9['u_coupon_count']
u9 = u9[['User_id','coupon_use_rate']]
# 某个用户线下优惠券购买率（优惠券购买/总购买）
u10 = u2.copy()
u10 = u10.merge(u3, on='User_id', how='left')
u10['u_buy_with_coupon'] = u10['u_buy_with_coupon'].fillna(0)
u10['coupon_purchase_rate'] = u10['u_buy_with_coupon']/u10['u_buy_count']
u10 = u10[['User_id','coupon_purchase_rate']]
# 某个用户线下领券-消费的平均时间间隔（天）
u11 = train_offline[((~train_offline['Date'].isna()) & (~train_offline['Date_received'].isna()))][['User_id','Purchase_Interval']].copy()
u11 = u11.groupby(['User_id'], as_index = False).mean()
u11.rename(columns = {'Purchase_Interval':'U_Purchase_Interval'}, inplace = True)
# 某一用户领取线上优惠券的次数 u1
u12 = train_online[(~train_online['Date_received'].isna())&(train_online['Action']==2)][['User_id']].copy()
u12['u_coupon_count_online'] = 1
u12 = u12.groupby(['User_id'], as_index = False).count()
# 某一用户线上购买的次数 u2
u13 = train_online[(~train_online['Date'].isna())&(train_online['Action']==1)][['User_id']].copy()
u13['u_buy_count_online'] = 1
u13 = u13.groupby(['User_id'], as_index = False).count()
# 某一用户使用线上优惠券的次数
u14 = train_online[(~train_online['Date'].isna())&(train_online['Action']==1) & (~train_online['Date_received'].isna())][['User_id']].copy()
u14['u_buy_with_coupon_online'] = 1
u14 = u14.groupby(['User_id'], as_index = False).count()
# 某一用户购买过的线上商店数
u15 = train_online[(~train_online['Date'].isna())&(train_online['Action']==1)][['User_id', 'Merchant_id']].copy()
u15 = u15.groupby(['User_id'], as_index = False).aggregate('nunique')
u15.rename(columns = {'Merchant_id':'u_merchant_count_online'}, inplace = True)
# 某一用户线上点击的次数
u16 = train_online[train_online['Action']==0][['User_id']].copy()
u16['u_click_count_online'] = 1
u16 = u16.groupby(['User_id'], as_index = False).count()
# 某个用户线上领券-消费的平均时间间隔（天）
u17 = train_online[((~train_online['Date'].isna())& (train_online['Action']==1) & (~train_online['Date_received'].isna()))][['User_id','Purchase_Interval']].copy()
u17 = u17.groupby(['User_id'], as_index = False).mean()
u17.rename(columns = {'Purchase_Interval':'U_Purchase_Interval_online'}, inplace = True)

In [ ]:
# merge all the features on key User_id
user_feature = pd.merge(u, u1, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u2, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u3, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u4, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u5, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u6, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u7, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u8, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u9, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u10, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u11, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u12, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u13, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u14, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u15, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u16, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u17, on = 'User_id', how = 'left')

In [ ]:
features = [i for i in user_feature.columns if i not in['User_id','u_min_distance','u_max_distance','u_mean_distance','u_median_distance']]
for feature in features:
  user_feature[feature] = user_feature[feature].fillna(0)
user_feature

,User_id,u_coupon_count,u_buy_count,u_buy_with_coupon,u_merchant_count,u_min_distance,u_max_distance,u_mean_distance,u_median_distance,coupon_use_rate,coupon_purchase_rate,Purchase_Interval,u_coupon_count_online,u_buy_count_online,u_buy_with_coupon_online,u_merchant_count_online,u_click_count_online,Purchase_Interval_online
0,1439408,5.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.2,0.333333,28.00,0.0,0.0,0.0,0.0,0.0,0.0
1,1832624,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,2029232,3.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.000000,0.00,0.0,2.0,0.0,0.0,0.0,0.0
3,2747744,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,196342,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539433,653784,0.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.000000,0.00,0.0,1.0,0.0,0.0,0.0,0.0
539434,3795296,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.000000,0.00,0.0,1.0,0.0,0.0,0.0,0.0
539435,188086,1.0,4.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.000000,0.00,0.0,1.0,0.0,0.0,33.0,0.0
539436,212662,5.0,9.0,4.0,0.0,2.0,6.0,4.0,4.0,0.8,0.444444,9.75,0.0,0.0,0.0,0.0,0.0,0.0


## 商家特征

In [ ]:
# key of merchant
m = train_offline[['Merchant_id']].copy().drop_duplicates()

# 某一商家券被领取总次数
m1 = train_offline[~train_offline['Date_received'].isna()][['Merchant_id']].copy()
m1['m_coupon_count'] = 1
m1 = m1.groupby(['Merchant_id'], as_index = False).count()
# 某一商家消费总次数
m2 = train_offline[~train_offline['Date'].isna()][['Merchant_id']].copy()
m2['m_sale_count'] = 1
m2 = m2.groupby(['Merchant_id'], as_index = False).count()
# 某一商家券被使用总次数
m3 = train_offline[(~train_offline['Date'].isna()) & (~train_offline['Date_received'].isna())][['Merchant_id']].copy()
m3['m_sale_with_coupon'] = 1
m3 = m3.groupby(['Merchant_id'], as_index = False).count()
# 某一商家购买距离的统计特征
mtmp = train_offline[(~train_offline['Date'].isna()) & (~train_offline['Date_received'].isna())][['Merchant_id', 'distance']].copy()
mtmp.replace(-1, np.nan, inplace = True)
m4 = mtmp.groupby(['Merchant_id'], as_index = False).min()
m4.rename(columns = {'distance':'m_min_distance'}, inplace = True)
m5 = mtmp.groupby(['Merchant_id'], as_index = False).max()
m5.rename(columns = {'distance':'m_max_distance'}, inplace = True)
m6 = mtmp.groupby(['Merchant_id'], as_index = False).mean()
m6.rename(columns = {'distance':'m_mean_distance'}, inplace = True)
m7 = mtmp.groupby(['Merchant_id'], as_index = False).median()
m7.rename(columns = {'distance':'m_median_distance'}, inplace = True)
# 某个商家线下领券-消费的平均时间间隔（天）
m8 = train_offline[((~train_offline['Date'].isna()) & (~train_offline['Date_received'].isna()))][['Merchant_id','Purchase_Interval']].copy()
m8 = m8.groupby(['Merchant_id'], as_index = False).mean()
m8.rename(columns = {'Purchase_Interval':'M_Purchase_Interval'}, inplace = True)
# 某一商家购买过的线下用户数
m9 = train_offline[(~train_offline['Date'].isna())][['Merchant_id', 'User_id']].copy()
m9 = m9.groupby(['Merchant_id'], as_index = True).aggregate('nunique')
m9 = m9.drop(columns='Merchant_id').reset_index()
m9.rename(columns = {'User_id':'m_user_count'}, inplace = True)

In [ ]:
merchant_feature = pd.merge(m, m1, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m2, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m3, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m4, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m5, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m6, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m7, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m8, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m9, on = 'Merchant_id', how = 'left')

In [ ]:
merchant_feature['m_coupon_use_rate'] = merchant_feature['m_sale_with_coupon'].astype('float')/merchant_feature['m_coupon_count'].astype('float')
merchant_feature['m_sale_with_coupon_rate'] = merchant_feature['m_sale_with_coupon'].astype('float')/merchant_feature['m_sale_count'].astype('float')

features = [i for i in merchant_feature.columns if i not in['Merchant_id','m_min_distance','m_max_distance','m_mean_distance','m_median_distance']]
for feature in features:
  merchant_feature[feature] = merchant_feature[feature].fillna(0)
merchant_feature.head()

,Merchant_id,m_coupon_count,m_sale_count,m_sale_with_coupon,m_min_distance,m_max_distance,m_mean_distance,m_median_distance,M_Purchase_Interval,m_user_count,m_coupon_use_rate,m_sale_with_coupon_rate
0,2632,43.0,21.0,4.0,0.0,1.0,0.750000,1.0,24.000000,9.0,0.093023,0.190476
1,4663,15191.0,1434.0,76.0,0.0,10.0,2.073529,1.0,10.921053,1056.0,0.005003,0.052999
2,3381,122834.0,21843.0,2487.0,0.0,10.0,1.652429,1.0,12.618818,12199.0,0.020247,0.113858
3,450,63042.0,12285.0,1461.0,0.0,10.0,0.892164,0.0,11.399042,3823.0,0.023175,0.118926
4,6459,16.0,20.0,0.0,NaN,NaN,NaN,NaN,0.000000,13.0,0.000000,0.000000


## 用户-商家特征

In [ ]:
# key of user and merchant
um = train_offline[['User_id', 'Merchant_id']].copy().drop_duplicates()

# 某一用户在某一商家交互的总次数
um1 = train_offline[['User_id', 'Merchant_id']].copy()
um1['um_count'] = 1
um1 = um1.groupby(['User_id', 'Merchant_id'], as_index = False).count()
# 某一用户在某一商家交易的总次数
um2 = train_offline[~train_offline['Date'].isna()][['User_id', 'Merchant_id']].copy()
um2['um_buy_count'] = 1
um2 = um2.groupby(['User_id', 'Merchant_id'], as_index = False).count()
# 某一用户在某一商家领取优惠券的总次数
um3 = train_offline[~train_offline['Date_received'].isna()][['User_id', 'Merchant_id']].copy()
um3['um_coupon_count'] = 1
um3 = um3.groupby(['User_id', 'Merchant_id'], as_index = False).count()
# 某一用户在某一商家使用优惠券的总次数
um4 = train_offline[(~train_offline['Date_received'].isna()) & (~train_offline['Date'].isna())][['User_id', 'Merchant_id']].copy()
um4['um_buy_with_coupon'] = 1
um4 = um4.groupby(['User_id', 'Merchant_id'], as_index = False).count()

In [ ]:
# merge all user merchant 
user_merchant_feature = pd.merge(um, um1, on = ['User_id','Merchant_id'], how = 'left')
user_merchant_feature = pd.merge(user_merchant_feature, um2, on = ['User_id','Merchant_id'], how = 'left')
user_merchant_feature = pd.merge(user_merchant_feature, um3, on = ['User_id','Merchant_id'], how = 'left')
user_merchant_feature = pd.merge(user_merchant_feature, um4, on = ['User_id','Merchant_id'], how = 'left')
user_merchant_feature = user_merchant_feature.fillna(0)

user_merchant_feature['um_buy_rate'] = user_merchant_feature['um_buy_count'].astype('float')/user_merchant_feature['um_count'].astype('float')
user_merchant_feature['um_coupon_use_rate'] = user_merchant_feature['um_buy_with_coupon'].astype('float')/user_merchant_feature['um_coupon_count'].astype('float')
user_merchant_feature['um_buy_with_coupon_rate'] = user_merchant_feature['um_buy_with_coupon'].astype('float')/user_merchant_feature['um_buy_count'].astype('float')
user_merchant_feature = user_merchant_feature.fillna(0)
user_merchant_feature.head()

,User_id,Merchant_id,um_count,um_buy_count,um_coupon_count,um_buy_with_coupon,um_buy_rate,um_coupon_use_rate,um_buy_with_coupon_rate
0,1439408,2632,6,3.0,4.0,1.0,0.5,0.25,0.333333
1,1439408,4663,1,0.0,1.0,0.0,0.0,0.00,0.000000
2,1832624,3381,1,0.0,1.0,0.0,0.0,0.00,0.000000
3,2029232,3381,1,0.0,1.0,0.0,0.0,0.00,0.000000
4,2029232,450,1,0.0,1.0,0.0,0.0,0.00,0.000000


## 整合所有特征

搞训练集

In [ ]:
train_offline = pd.merge(train_offline, user_feature, on='User_id', how='left')
train_offline = pd.merge(train_offline, merchant_feature, on='Merchant_id', how='left')
train_offline = pd.merge(train_offline, user_merchant_feature, on=['User_id','Merchant_id'], how='left')

搞测试集

In [ ]:
test_offline = pd.merge(test_offline, user_feature, on='User_id', how='left')
test_offline = pd.merge(test_offline, merchant_feature, on='Merchant_id', how='left')
test_offline = pd.merge(test_offline, user_merchant_feature, on=['User_id','Merchant_id'], how='left')

In [ ]:
print('test_offline dataset has {} rows and {} columns.'.format(test_offline.shape[0],test_offline.shape[1]))
print('train_offline dataset has {} rows and {} columns.'.format(train_offline.shape[0],train_offline.shape[1]))

test_offline dataset has 113640 rows and 55 columns.
train_offline dataset has 1754884 rows and 56 columns.


In [ ]:
test_features = test_offline.columns
trainfeatures = train_offline.columns

for feature in trainfeatures:
  if feature not in test_features:
    print(feature)

label


In [ ]:
train_offline.rename(columns={'Purchase_Interval_x':'Purchase_Interval', 'Purchase_Interval_y':'U_Purchase_Interval'},inplace=True)
test_offline.rename(columns={'Purchase_Interval':'U_Purchase_Interval'},inplace=True)

In [ ]:
train_offline = train_offline.drop(columns=['Purchase_Interval','Date'])

In [ ]:
train_offline.to_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/train.csv', index=0)
test_offline.to_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/test.csv', index=0)